In [ ]:
# Импортирование модулей сервиса
from NLU_classifier import NLU_Classifier
from NerExtractor   import NerExtractor
from VicunaBot      import VicunaBot

import sqlite3

class DBConnection:
    """
    Класс для управления подключением к базе данных SQLite.

    Атрибуты:
        db_path (str): Путь к файлу базы данных SQLite.
    """

    def __init__(self, db_path):
        """Инициализация с путём к файлу базы данных."""
        self.db_path = db_path

    def create_connection(self):
        """Создание соединения с базой данных."""
        conn = None
        try:
            conn = sqlite3.connect(self.db_path)
        except Error as e:
            print(f"Ошибка при подключении к базе данных: {e}")
        return conn

    def close_connection(self, conn):
        """Закрытие соединения с базой данных."""
        if conn:
            conn.close()

    def execute(self, query, parameters=()):
        """
        Выполнение SQL-запроса с возможностью изменения данных.
        """
        conn = self.create_connection()
        if conn is not None:
            try:
                cursor = conn.cursor()
                cursor.execute(query, parameters)
                conn.commit()
            except Error as e:
                print(f"Ошибка при выполнении SQL: {e}")
            finally:
                self.close_connection(conn)

    def query(self, query, parameters=()):
        """
        Выполнение SQL-запроса и возврат результатов.
        """
        conn = self.create_connection()
        results = None
        if conn is not None:
            try:
                cursor = conn.cursor()
                cursor.execute(query, parameters)
                results = cursor.fetchall()
            except Error as e:
                print(f"Ошибка при выполнении SQL: {e}")
            finally:
                self.close_connection(conn)
        return results

class DialogueHistory:
    """
    Класс для управления историей диалогов в базе данных.

    Атрибуты:
        db (DBConnection): Экземпляр класса для подключения к базе данных.
    """

    def __init__(self, db_connection):
        """Инициализация с объектом подключения к базе данных.

        :param vicuna_bot: объект VicunaBot для доступа к глобальному состоянию бота
        """

        self.db = db_connection

    def initialize_dialogue(self, user_id):
        """
        Инициализация нового диалога для пользователя.

        Параметры:
            user_id (int): Уникальный идентификатор пользователя.
        """
        self.db.execute(
            "INSERT INTO Dialogues (user_id, history) VALUES (?, ?) ON CONFLICT(user_id) DO NOTHING",
            (user_id, "")
        )

    def add_to_history(self, user_id, input_text):
        """
        Добавляет запись о входящем сообщении пользователя в базу данных.

        :param user_id: int - уникальный идентификатор пользователя.
        :param input_text: str - текст входящего сообщения пользователя.
        """
        current_time = datetime.datetime.now()
        self.db.execute(
            "INSERT INTO Dialogues (user_id, time, input_text) VALUES (?, ?, ?)",
            (user_id, current_time, input_text)
        )

    def get_history(self, user_id):
        """
        Получение истории диалогов пользователя.

        Параметры:
            user_id (int): Уникальный идентификатор пользователя.

        Возвращает:
            str: История диалогов пользователя.
        """
        result = self.db.query("SELECT history FROM Dialogues WHERE user_id = ?", (user_id,))
        return result[0][0] if result else ""

    def dump_conversation(self, user_id):
        """
        Сохраняет всю историю диалога в базу данных после завершения сессии.

        :param user_id: int - уникальный идентификатор пользователя.
        """
        rows = self.db.query(
            "SELECT input_text, response FROM Dialogues WHERE user_id = ?",
            (user_id,)
        )
        # Формирование строки истории для сохранения
        history = "\n".join(f"USER: {row[0]} ASSISTANT: {row[1]}" for row in rows)
        # Обновление записи истории диалога
        self.db.execute(
            "UPDATE Dialogues SET history = ? WHERE user_id = ?",
            (history, user_id)
        )

    def update_conversation(self, user_id, history, input_text, response):
        """
        Обновление информации о диалоге в базе данных.

        Параметры:
            user_id (int): Уникальный идентификатор пользователя.
            history (DialogueHistory): Объект истории диалогов, содержащий вводы и ответы пользователя.
            input_text (str): Текст последнего ввода пользователя.
            response (str): Текст последнего ответа системы.

        Описание: Обновляет информацию о текущем диалоге в базе данных.
        """
        try:
            # Получение ID последнего разговора для конкретного пользователя из базы данных
            last_conversation_id = self.db.query(
                'SELECT id FROM conversations WHERE user_id = ? ORDER BY id DESC LIMIT 1', (user_id,)
            )[0][0]

            history_text = ' '.join([
                "USER: " + u + " ASSISTANT: " + ai + "</s>"
                for u, ai in zip(history.get_user_inputs(), history.get_assistant_responses())
            ])

            # Обновление ответа, истории и входных данных последнего завершенного разговора
            self.db.execute(
                'UPDATE conversations SET response = ?, history = ?, input = ? WHERE user_id = ? AND id = ?',
                (response, history_text, input_text, user_id, last_conversation_id)
            )
        except Exception as e:
            print(f"Ошибка при обновлении диалога в базе данных: {e}")

    def user_init(self):
        """
        Инициализация пользователя и его диалоговой истории.

        Описание: Создает и настраивает диалоговую историю для нового пользователя.
        """
        gen_kwargs = {  "max_new_tokens" : 2048,
                            # "n_ctx" : 2048,
                            # 'max_tokens' : 512,
                            # "max_length": 512,      # The maximum length of the generated text in tokens
                            # "num_beams": 5,         # The number of beams for beam search decoding (set to 1 for greedy decoding)
                            "temperature": 0.9,       # Controls the randomness of the generation; higher values make it more random
                            "do_sample": True,        # Whether to use sampling for text generation (True) or not (False)
                            # "top_k": 50,            # Controls the number of highest probability tokens to consider for sampling
                            # "top_p": 0.95,          # Controls the cumulative probability threshold for sampling (nucleus sampling)
                            }
        # gen_kwargs = dict(max_new_tokens=512)

        # Обновление аргументов генерации для VicunaBot
        self.vicuna_bot.gen_kwargs = gen_kwargs

        # Использование существующего экземпляра VicunaBot
        chat = self.vicuna_bot

        dialogue_history = self.DialogueHistory()
        return [chat, '', dialogue_history]

class ConversationManager:
    """
    Класс для управления диалогами, интегрирующий работу с историей и базой данных.

    Атрибуты:
        db_connection (DBConnection): Экземпляр класса для подключения к базе данных.
        dialogue_history (DialogueHistory): Экземпляр класса для управления историей диалогов.
    """

    def __init__(self, db_path, vicuna_bot: VicunaBot):
        """Инициализация с путём к файлу базы данных."""
        self.db_connection = DBConnection(db_path)
        self.dialogue_history = DialogueHistory(self.db_connection)
        self.vicuna_bot = VicunaBot()

    def start_conversation(self, user_id):
        """
        Начало новой беседы с инициализацией истории для пользователя.

        Параметры:
            user_id (int): Уникальный идентификатор пользователя.
        """
        self.dialogue_history.initialize_dialogue(user_id)

    def add_message(self, user_id, message):
        """
        Добавление сообщения в историю диалога пользователя.

        Параметры:
            user_id (int): Уникальный идентификатор пользователя.
            message (str): Сообщение для добавления в историю.
        """
        self.dialogue_history.add_to_history(user_id, message)

    def get_conversation_history(self, user_id):
        """
        Получение полной истории диалога пользователя.

        Параметры:
            user_id (int): Уникальный идентификатор пользователя.

        Возвращает:
            str: Полная история диалогов пользователя.
        """
        return self.dialogue_history.get_history(user_id)

    def handle_query(self, user_id, query, domain):
        """
        Обработка запроса от пользователя.

        Параметры:
            user_id (int): Уникальный идентификатор пользователя.
            query   (str): Запрос пользователя.
            domain  (str): Тематика запроса.
        """
        # Создаем экземпляр NLU_Classifier
        nlu_classifier = NLU_Classifier()

        # Передаем запрос и в NLU_Classifier для обработки
        is_relevant, response = nlu_classifier.get_context(query)

        # Проверка релевантности запроса
        if is_relevant:
            # Обработка релевантного запроса
            self.process_relevant_query(user_id, query)
        else:
            # Отправка стандартного ответа, если запрос не релевантен
            self.send_default_response(user_id)

    def send_default_response(self, user_id):
        """
        Отправка стандартного ответа пользователю.

        Параметры:
            user_id (int): Уникальный идентификатор пользователя.
        """
        default_response = "Я искусственный интеллект, работающий с осмысленным текстом. Поэтому здесь вам помочь не могу."
        self.add_message(user_id, default_response)
        # Здесь должен быть код, отправляющий сообщение пользователю через Телеграм бота

    def process_relevant_query(self, user_id, query):
        """
        Обработка релевантного запроса.

        Параметры:
            user_id (int): Уникальный идентификатор пользователя.
            query   (str): Запрос пользователя.
        """
        # Здесь должен быть код, который отправляет запрос на обработку в service bot
        pass